In [1]:
import os
import sys #current as of 9/26/2015
spark_home = os.environ['SPARK_HOME'] = \
   '/opt/spark16'
    
sys.path.insert(0,os.path.join(spark_home,'python'))
sys.path.insert(0,os.path.join(spark_home,'python/lib/py4j-0.8.2.1-src.zip'))
execfile(os.path.join(spark_home,'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.6.1
      /_/

Using Python version 2.7.10 (default, Oct 23 2015 19:19:21)
SparkContext available as sc, SQLContext available as sqlContext.


In [2]:
DATA_DIR = 'Data/'
TRAIN_FILE = 'train.csv.gz'
TEST_FILE = 'test.csv.gz'
PRODUCT_FILE = 'producto_tabla.csv.gz'
CLIENT_FILE = 'cliente_table.csv.gz'

In [8]:
train, val = sc.textFile(DATA_DIR+TRAIN_FILE).randomSplit([0.8,0.2])
train.cache()
val.cache()

test = sc.textFile(DATA_DIR+TRAIN_FILE).cache()

In [9]:
train.count()

59347356

In [13]:
import hashlib
import numpy as np
from math import log
from collections import defaultdict
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.regression import LabeledPoint, LinearRegressionWithSGD


def hashFunction(numBuckets, rawFeats, printMapping=False):
    """Calculate a feature dictionary for an observation's features based on hashing.

    Note:
        Use printMapping=True for debug purposes and to better understand how the hashing works.

    Args:
        numBuckets (int): Number of buckets to use as features.
        rawFeats (list of (int, str)): A list of features for an observation.  Represented as
            (featureID, value) tuples.
        printMapping (bool, optional): If true, the mappings of featureString to index will be
            printed.

    Returns:
        dict of int to float:  The keys will be integers which represent the buckets that the
            features have been hashed to.  The value for a given key will contain the count of the
            (featureID, value) tuples that have hashed to that key.
    """
    mapping = {}
    for ind, category in rawFeats:
        featureString = category + str(ind)
        mapping[featureString] = int(int(hashlib.md5(featureString).hexdigest(), 16) % numBuckets)
    if(printMapping): print mapping
    sparseFeatures = defaultdict(float)
    for bucket in mapping.values():
        sparseFeatures[bucket] += 1.0
    return dict(sparseFeatures)


def parseHashPoint(point, numBuckets, sep=','):
    """Create a LabeledPoint for this observation using hashing.

    Args:
        point (str): A comma separated string where the first value is the label and the rest are
            features.
        numBuckets: The number of buckets to hash to.
        sep: The separator of the input features.

    Returns:
        LabeledPoint: A LabeledPoint with a label (0.0 or 1.0) and a SparseVector of hashed
            features.
    """
    data = point.split(sep)
    label = data[-1]
    features = data[1:-5]
    indexed_features = [(i, feature) for i, feature in enumerate(features)]
    return LabeledPoint(label, SparseVector(numBuckets, hashFunction(numBuckets, indexed_features)))


In [11]:
hashTrainData = train.map(lambda x: parseHashPoint(x, 2**15))
hashValData = val.map(lambda x: parseHashPoint(x, 2**15))

In [ ]:
lr_model = LinearRegressionWithSGD.train(
    data=hashTrainData
    , iterations=200
    , step=1.0
    , miniBatchFraction=0.5
    , regType='l2'
    , regParam=0.1
    , convergenceTol=1e-5
)